# Bank Prospect Data cleaning project with spark Dataframes

In [7]:
#pip install pyspark

In [42]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #here, i launch spark with all cpu cores i've...

## Read the CSV file into a DataFrame

In [9]:
bankProspectsDF = spark.read.csv("bank_prospects.csv",header=True)

In [10]:
bankProspectsDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



## Remove the record with "unknow" value in country column

In [11]:
bankProspectsDF1 = bankProspectsDF.filter(bankProspectsDF['country'] != "unknown")

In [12]:
bankProspectsDF1.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



##  Cast the String datatype to Integer/Float

In [13]:
bankProspectsDF1.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [14]:
from pyspark.sql.types import IntegerType,FloatType

In [15]:
bankProspectsDF2 = bankProspectsDF1.withColumn("age", bankProspectsDF1["age"].cast(IntegerType())).withColumn("salary", bankProspectsDF1["salary"].cast(FloatType()))


In [16]:
bankProspectsDF2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



## Replace Age and Salary with average values of their respective column

<h3>import mean from sql.fuctions</h3>

In [18]:
from pyspark.sql.functions import mean

### Calculate "mean" value of the age

In [19]:
mean_age_val = bankProspectsDF2.select(mean(bankProspectsDF2['age'])).collect()

In [20]:
type(mean_age_val)

list

In [21]:
mean_age_val

[Row(avg(age)=22.11111111111111)]

In [27]:
mean_age = mean_age_val[0][0]

In [28]:
mean_age

22.11111111111111

### Calculate mean salary value

In [30]:
mean_salary_val = bankProspectsDF2.select(mean(bankProspectsDF2['salary'])).collect()

In [31]:
mean_salary = mean_salary_val[0][0]

In [32]:
mean_salary

31875.0

### Replace missing age with average value

In [33]:
bankProspectsDF2.show()

+----+-------+------+-------+---------+
| age| salary|Gender|Country|Purchased|
+----+-------+------+-------+---------+
|  18|20000.0|  Male|Germany|        N|
|  19|22000.0|Female| France|        N|
|  20|24000.0|Female|England|        N|
|  21|   null|  Male|England|        N|
|  22|50000.0|  Male| France|        Y|
|  23|35000.0|Female|England|        N|
|  24|   null|  Male|Germany|        N|
|  25|32000.0|Female| France|        Y|
|null|35000.0|  Male|Germany|        N|
|  27|37000.0|Female| France|        N|
+----+-------+------+-------+---------+



In [36]:
bankbankProspectsDF3 = bankProspectsDF2.na.fill(mean_age,["age"])

In [37]:
bankbankProspectsDF3.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|   null|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|   null|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



### Replace missing age with salary value

In [38]:
bankbankProspectsDF4 = bankbankProspectsDF3.na.fill(mean_salary,["salary"])

In [39]:
bankbankProspectsDF4.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|31875.0|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|31875.0|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



In [40]:
bankbankProspectsDF4.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = false)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



## Write the transformed file to a new csv file 

In [41]:
bankbankProspectsDF4.write.format("csv").save("bank_prospects_transformed")

In this path new cleaned data will resides :------> bank_prospects_transformed/part-00000-457e6249-a16a-468f-988a-149fddea139f-c000.csv

<h2>Done !!</h2>
<h3>Here we clean the data and handle misssing values with the mean value of that particular column and fill wih it. now this data is ready for further Analysis.</h3>